# Access an ASAM ODS EXD-API Plugin

## Prepare Python Environment to Access GRPC Service

In [8]:
try:
    import ods_external_data_pb2_grpc as ods_external_data_pb2_grpc_check_import
except:
    !python -m pip install --upgrade pip
    !python -m pip install grpcio
    !python -m pip install grpcio-tools
    ## To update stubs download protos and run this lines
    #!python -m grpc_tools.protoc --proto_path=. --pyi_out=. --python_out=. ods.proto
    #!python -m grpc_tools.protoc -I. --pyi_out=. --python_out=. --grpc_python_out=. ods_external_data.proto

In [9]:
import os
import pathlib

import grpc
from google.protobuf.json_format import MessageToJson

import ods_pb2
import ods_external_data_pb2
import ods_external_data_pb2_grpc

## EXD-API

The EXD-API plugin is running as a RPC service at a given URL.
Running `exd_api_server.py`´will run the plugin at the given URL.

In [10]:
exd_api_plugin_url = "localhost:50051"

## Import Phase

We will open a MDF4 file using the EXD-API and extract the internal structure of the file to import it into the ASAM ODS server.

In [11]:
data_file_path = os.path.abspath("data/simple.mf4")
if not os.path.exists(data_file_path):
    raise Exception("Data file is missing")

In [12]:
import_file_url = pathlib.Path(data_file_path).as_uri()
import_file_parameters = ""
print(import_file_url)

# Will be filled from Structure
access_file_url = None
access_file_parameters = None

file:///c:/Users/AKR/github/asam_ods_exd_api_mdf4/data/simple.mf4


### Extract Infos from Structure

The structure contains infos about groups and channels to create corresponding measurements, submatrices and measurement_quantities

In [13]:
with grpc.insecure_channel(exd_api_plugin_url) as channel:
    stub = ods_external_data_pb2_grpc.ExternalDataReaderStub(channel)

    # import file into ASAM ODS Server physical storage
    import_identifier = ods_external_data_pb2.Identifier(url=import_file_url, parameters=import_file_parameters)

    import_handle = stub.Open(import_identifier)
    try:
        structure = stub.GetStructure(ods_external_data_pb2.StructureRequest(handle=import_handle))
        print(MessageToJson(structure))

        access_file_url = structure.identifier.url
        access_file_parameters = structure.identifier.parameters

        for group in structure.groups:
            group_id = group.id
            for channel in group.channels:
                channel_id = channel.id
    finally:
        stub.Close(import_handle)

{
  "identifier": {
    "url": "file:///c:/Users/AKR/github/asam_ods_exd_api_mdf4/data/simple.mf4",
    "parameters": ""
  },
  "name": "simple.mf4",
  "groups": [
    {
      "id": "0",
      "name": "",
      "totalNumberOfChannels": "10",
      "numberOfRows": "563",
      "channels": [
        {
          "id": "0",
          "name": "time",
          "dataType": "DT_DOUBLE",
          "unitString": "s",
          "attributes": {
            "variables": {
              "description": {
                "stringArray": {
                  "values": [
                    ""
                  ]
                }
              }
            }
          }
        },
        {
          "id": "1",
          "name": "abs_time",
          "dataType": "DT_LONGLONG",
          "unitString": "ns",
          "attributes": {
            "variables": {
              "description": {
                "stringArray": {
                  "values": [
                    "absolute timestamp in nano seco

## Access Bulk Data

With the stored information the ASAM ODS server can access the bulk data from the EXD-API plugin

In [14]:
with grpc.insecure_channel(exd_api_plugin_url) as channel:
    stub = ods_external_data_pb2_grpc.ExternalDataReaderStub(channel)

    # info from physical storage
    access_group_id = 0
    access_channel_ids = [0, 1, 2]
    access_identifier = ods_external_data_pb2.Identifier(url=access_file_url, parameters=access_file_parameters)

    # open bulk access
    access_handle = stub.Open(access_identifier)
    try:
        request = ods_external_data_pb2.ValuesRequest(
            handle=access_handle, group_id=access_group_id, channel_ids=access_channel_ids
        )

        # read first chunk
        request.start = 0
        request.limit = 3
        values = stub.GetValues(request)
        print(MessageToJson(values))

        # read second chunk
        request.start = 3
        request.limit = 10
        values = stub.GetValues(request)
        print(MessageToJson(values))

    finally:
        stub.Close(access_handle)

{
  "channels": [
    {
      "id": "0",
      "values": {
        "dataType": "DT_DOUBLE",
        "unitId": "0",
        "doubleArray": {
          "values": [
            0.0,
            17905500.0,
            30515300.0
          ]
        }
      }
    },
    {
      "id": "1",
      "values": {
        "dataType": "DT_LONGLONG",
        "unitId": "0",
        "longlongArray": {
          "values": [
            "1686082945335777300",
            "1686082945353682800",
            "1686082945366292600"
          ]
        }
      }
    },
    {
      "id": "2",
      "values": {
        "dataType": "DT_LONG",
        "unitId": "0",
        "longArray": {
          "values": [
            112,
            112,
            112
          ]
        }
      }
    }
  ]
}
{
  "channels": [
    {
      "id": "0",
      "values": {
        "dataType": "DT_DOUBLE",
        "unitId": "0",
        "doubleArray": {
          "values": [
            48317800.0,
            66109300.0,
      